In [88]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.special import logsumexp
from matplotlib import pyplot as plt


def get_random_psd(n):
    x = np.random.normal(0, 1, size=(n, n))
    return np.dot(x, x.transpose())


def initialize_random_params():
    params = {'phi': np.random.uniform(0, 1),
              'mu0': np.random.normal(0, 1, size=(2,)),
              'mu1': np.random.normal(0, 1, size=(2,)),
              'sigma0': get_random_psd(2),
              'sigma1': get_random_psd(2)}
    return params


def learn_params(x_labeled, y_labeled):
    n = x_labeled.shape[0]
    phi = x_labeled[y_labeled == 1].shape[0] / n
    mu0 = np.sum(x_labeled[y_labeled == 0], axis=0) / x_labeled[y_labeled == 0].shape[0]
    mu1 = np.sum(x_labeled[y_labeled == 1], axis=0) / x_labeled[y_labeled == 1].shape[0]
    sigma0 = np.cov(x_labeled[y_labeled == 0].T, bias= True)
    sigma1 = np.cov(x_labeled[y_labeled == 1].T, bias=True)
    return {'phi': phi, 'mu0': mu0, 'mu1': mu1, 'sigma0': sigma0, 'sigma1': sigma1}


def e_step(x, params):
    np.log([stats.multivariate_normal(params["mu0"], params["sigma0"]).pdf(x),
            stats.multivariate_normal(params["mu1"], params["sigma1"]).pdf(x)])
    log_p_y_x = np.log([1-params["phi"], params["phi"]])[np.newaxis, ...] + \
                np.log([stats.multivariate_normal(params["mu0"], params["sigma0"]).pdf(x),
            stats.multivariate_normal(params["mu1"], params["sigma1"]).pdf(x)]).T
    log_p_y_x_norm = logsumexp(log_p_y_x, axis=1)
    return log_p_y_x_norm, np.exp(log_p_y_x - log_p_y_x_norm[..., np.newaxis])



def m_step(x, params):
    total_count = x.shape[0]
    _, heuristics = e_step(x, params)
    heuristic0 = heuristics[:, 0]
    heuristic1 = heuristics[:, 1]
    sum_heuristic1 = np.sum(heuristic1)
    sum_heuristic0 = np.sum(heuristic0)
    phi = (sum_heuristic1/total_count)
    mu0 = (heuristic0[..., np.newaxis].T.dot(x)/sum_heuristic0).flatten()
    mu1 = (heuristic1[..., np.newaxis].T.dot(x)/sum_heuristic1).flatten()
    diff0 = x - mu0
    sigma0 = diff0.T.dot(diff0 * heuristic0[..., np.newaxis]) / sum_heuristic0
    diff1 = x - mu1
    sigma1 = diff1.T.dot(diff1 * heuristic1[..., np.newaxis]) / sum_heuristic1
    params = {'phi': phi, 'mu0': mu0, 'mu1': mu1, 'sigma0': sigma0, 'sigma1': sigma1}
    return params


def get_avg_log_likelihood(x, params):
    loglikelihood, _ = e_step(x, params)
    return np.mean(loglikelihood)


def run_em(x, params):
    avg_loglikelihoods = []
    while True:
        avg_loglikelihood = get_avg_log_likelihood(x, params)
        avg_loglikelihoods.append(avg_loglikelihood)
        if len(avg_loglikelihoods) > 2 and abs(avg_loglikelihoods[-1] - avg_loglikelihoods[-2]) < 0.0001:  # break condition when the loglikelihood is not changing
            break
        params = m_step(x, params)
    print("EM algorithm converged.")
    print("Final Parameters:")
    print(f"\tphi: {params['phi']}")
    print(f"\tmu_0: {params['mu0']}")
    print(f"\tmu_1: {params['mu1']}")
    print(f"\tsigma_0:\n{params['sigma0']}")
    print(f"\tsigma_1:\n{params['sigma1']}")
    print(x.shape)

    _, posterior = e_step(x, params)
    forecasts = np.argmax(posterior, axis=1)
    return forecasts, posterior, avg_loglikelihoods




In [89]:
# Provide the path to your text file
file_path = "https://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/HW2/2gaussian.txt"

# Read the data into a dataframe
df = pd.read_csv(file_path, sep=" ", header=None, names=['x1', 'x2'])
x = df.values

### Results for 2 gaussian

In [90]:
    
# Unsupervised learning
print("Learned Parameter For 2 gaussian: ")
random_params = initialize_random_params()
unsupervised_forecasts, unsupervised_posterior, unsupervised_loglikelihoods = run_em(x, random_params)
print("total steps: ", len(unsupervised_loglikelihoods))
    # Semi-supervised learning
"""source: https://github.com/VXU1230/Medium-Tutorials/tree/master/em"""

Learned Parameter: 
EM algorithm converged.
Final Parameters:
	phi: 0.667517710695926
	mu_0: [2.98061058 3.05387953]
	mu_1: [7.005954   3.97901935]
	sigma_0:
[[0.98826364 0.02991258]
 [0.02991258 2.9484209 ]]
	sigma_1:
[[0.98752928 0.50473472]
 [0.50473472 1.0072154 ]]
(6000, 2)
total steps:  20


'source: https://github.com/VXU1230/Medium-Tutorials/tree/master/em'

In [91]:
cluster_0_data = x[unsupervised_forecasts==0]
cluster_1_data = x[unsupervised_forecasts==1]

In [92]:
# Compute the mean of each cluster
mean_cluster_0 = np.mean(cluster_0_data, axis=0)
mean_cluster_1 = np.mean(cluster_1_data, axis=0)

# Compute the covariance matrix of each cluster
cov_cluster_0 = np.cov(cluster_0_data.T)
cov_cluster_1 = np.cov(cluster_1_data.T)

print(f"mean_cluster_0 = {mean_cluster_0}")
print(f"mean_cluster_1 = {mean_cluster_1}")
print(f"cov_cluster_0 = {cov_cluster_0}")
print(f"cov_cluster_1 = {cov_cluster_1}")

mean_cluster_0 = [2.95292525 3.04194494]
mean_cluster_1 = [7.00972229 3.98264004]
cov_cluster_0 = [[0.90780697 0.01158188]
 [0.01158188 2.97341593]]
cov_cluster_1 = [[0.95711426 0.48533666]
 [0.48533666 0.99173716]]


## B

In [94]:
# Provide the path to your text file
file_path = "https://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/HW2/3gaussian.txt"

# Read the data into a dataframe
df = pd.read_csv(file_path, sep=" ", header=None, names=['x1', 'x2'])
x = df.values